## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np
# Import API key
os.chdir("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dedza,MW,-14.3779,34.3332,63.66,42,7,1.36,clear sky
1,1,Ola,RU,59.5833,151.2833,49.26,78,93,5.97,overcast clouds
2,2,San Patricio,US,28.0170,-97.5169,91.81,63,98,7.36,overcast clouds
3,3,Saint George,US,37.1041,-113.5841,100.87,11,1,3.00,clear sky
4,4,Khatanga,RU,71.9667,102.5000,40.24,94,100,9.66,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Arawa,PG,-6.2132,155.5367,78.49,82,72,3.13,broken clouds
10,10,Jamestown,US,42.0970,-79.2353,67.59,85,40,1.99,scattered clouds
13,13,Mitsamiouli,KM,-11.3847,43.2844,76.80,79,89,4.74,overcast clouds
15,15,Albany,US,42.6001,-73.9662,73.45,90,100,9.98,light rain
16,16,Sumbe,AO,-11.2061,13.8437,71.89,88,90,3.91,light rain
18,18,Vaini,TO,-21.2000,-175.2000,78.96,94,75,8.05,broken clouds
19,19,Saint-Philippe,RE,-21.3585,55.7679,70.21,82,25,7.92,scattered clouds
23,23,Lorengau,PG,-2.0226,147.2712,77.38,91,100,16.33,heavy intensity rain
25,25,Salvador,SV,13.8333,-88.9167,82.44,78,36,4.61,scattered clouds
30,30,Puerto Cabezas,NI,14.0351,-83.3888,85.96,74,40,6.91,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                349
City                   349
Country                347
Lat                    349
Lng                    349
Max Temp               349
Humidity               349
Cloudiness             349
Wind Speed             349
Current Description    349
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [10]:
preferred_cities_df.count()

City_ID                347
City                   347
Country                347
Lat                    347
Lng                    347
Max Temp               347
Humidity               347
Cloudiness             347
Wind Speed             347
Current Description    347
dtype: int64

In [11]:
clean_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Arawa,PG,78.49,broken clouds,-6.2132,155.5367,
10,Jamestown,US,67.59,scattered clouds,42.0970,-79.2353,
13,Mitsamiouli,KM,76.80,overcast clouds,-11.3847,43.2844,
15,Albany,US,73.45,light rain,42.6001,-73.9662,
16,Sumbe,AO,71.89,light rain,-11.2061,13.8437,
18,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
19,Saint-Philippe,RE,70.21,scattered clouds,-21.3585,55.7679,
23,Lorengau,PG,77.38,heavy intensity rain,-2.0226,147.2712,
25,Salvador,SV,82.44,scattered clouds,13.8333,-88.9167,
30,Puerto Cabezas,NI,85.96,scattered clouds,14.0351,-83.3888,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [16]:
hotel_df.count()

City                   347
Country                347
Max Temp               347
Current Description    347
Lat                    347
Lng                    347
Hotel Name             347
dtype: int64

In [24]:
nan_value = float("NaN")
hotel_df.replace("",nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

In [25]:
hotel_df.count()

City                   329
Country                329
Max Temp               329
Current Description    329
Lat                    329
Lng                    329
Hotel Name             329
dtype: int64

In [26]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Arawa,PG,78.49,broken clouds,-6.2132,155.5367,Coastline Guest House
10,Jamestown,US,67.59,scattered clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
13,Mitsamiouli,KM,76.80,overcast clouds,-11.3847,43.2844,Foyer ADM
16,Sumbe,AO,71.89,light rain,-11.2061,13.8437,Hotel Ritz Sumbe
18,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
19,Saint-Philippe,RE,70.21,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
23,Lorengau,PG,77.38,heavy intensity rain,-2.0226,147.2712,Lorengau Harbourside Hotel
25,Salvador,SV,82.44,scattered clouds,13.8333,-88.9167,Chico's Ranch
30,Puerto Cabezas,NI,85.96,scattered clouds,14.0351,-83.3888,Cabaña Caribe
31,Hithadhoo,MV,80.19,light rain,-0.6000,73.0833,Scoop Guest House


In [28]:
clean_hotel_df = hotel_df

In [29]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))